### Setup Ambiente

Neste vídeo, explicamos como fazer o fine-tuning de um modelo de forma local, utilizando o Google Colab. O Google Colab é um laboratório de colaboração gratuito do Google que permite construir modelos e usar GPUs gratuitamente. Mostramos como conectar o Colab com o Hugging Face para salvar e compartilhar modelos. Também demonstramos como criar um Access Token no Hugging Face para permitir a conexão entre as plataformas. Ao final, conectamos com sucesso ao Hugging Face.

In [1]:
from huggingface_hub import notebook_login
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding, TrainingArguments, Trainer, AutoModelForSequenceClassification, pipeline
import numpy as np
import evaluate

import os

In [2]:
notebook_login()

### Upload de arquivos

Neste vídeo, é mostrado como construir um modelo de Fine-tuning. São instaladas bibliotecas, feito o upload de datasets e explicado o processo de junção de arquivos para treinamento. O instrutor demonstra como carregar os datasets, mostrando a quantidade de linhas de input e output de cada um. É destacada a importância de unir os arquivos de treino e validação em um só para o Fine-tuning do BERT.

In [ ]:
dataset = load_dataset("json", data_files={"train": "train_aws.jsonl", "test": "validation_aws.jsonl"})

dataset = dataset.map(
    lambda examples: {
        "input": examples["prompt"],
        "output": examples["completion"],
    },

    remove_columns=["prompt", "completion"],
)
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 453
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 114
    })
})

1. `load_dataset("json", data_files={"train": "train_aws.jsonl", "test": "validation_aws.jsonl"})`:
	* Carrega um conjunto de dados em formato JSON a partir de dois arquivos: `train_aws.jsonl` e `validation_aws.jsonl`.
	* O parâmetro `data_files` especifica os nomes dos arquivos que contêm os dados de treinamento e teste.
	* O resultado é um objeto `dataset` que contém os dados carregados.
2. `dataset.map(...)`:
	* Aplica uma transformação aos dados carregados.
	* A transformação é definida por uma função lambda que recebe um exemplo de dado (`examples`) como entrada.
	* A função lambda retorna um novo exemplo de dado com as seguintes características:
		+ `input`: o valor do campo `prompt` do exemplo original.
		+ `output`: o valor do campo `completion` do exemplo original.
	* O parâmetro `remove_columns` especifica que os campos `prompt` e `completion` devem ser removidos do exemplo original após a transformação.
3. O resultado da transformação é atribuído novamente ao objeto `dataset`.

Em resumo, o código carrega um conjunto de dados em formato JSON, transforma os dados para ter apenas os campos `input` e `output`, e remove os campos originais `prompt` e `completion`. O resultado é um conjunto de dados pronto para ser usado em um modelo de linguagem.

### Preparando Dataset

Neste trecho da aula, começamos a construir modelos utilizando o BERT UNCASED. Criamos variáveis como CHECKPOINT e TOKENIZER para treinar o modelo. A função TRANSFORM LABELS é criada para ajustar as classes do dataset. A função TOKENIZER Function é definida para tokenizar o texto. Em seguida, é feita a tokenização do dataset e a transformação dos labels. Um erro é corrigido no código para garantir que o retorno esteja na posição correta.

In [5]:
checkpoint = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

mapDict = {
    "No Hate Speech": 0,
    "Hate Speech": 1,
}

def transform_labels(label):
    label = label["output"]
    result = []
    for l in label:
        result.append(mapDict[l])
    return {"label": result}


def tokenize_function(example):
    return tokenizer(example["input"], padding=True, truncation=True)

1. `checkpoint = "bert-base-uncased"`:
	* Define uma variável `checkpoint` com o valor `"bert-base-uncased"`.
	* Esse valor é o nome de um modelo de linguagem BERT pré-treinado, que é um modelo de linguagem baseado em transformer que foi treinado em um grande conjunto de texto não supervisionado.
	* O sufixo `"uncased"` indica que o modelo não diferencia entre maiúsculas e minúsculas.
2. `tokenizer = AutoTokenizer.from_pretrained(checkpoint)`:
	* Carrega um tokenizer associado ao modelo BERT definido na variável `checkpoint`.
	* O tokenizer é um objeto que é responsável por dividir o texto em tokens, que são as unidades básicas de processamento do modelo de linguagem.
	* O método `from_pretrained` é usado para carregar o tokenizer pré-treinado associado ao modelo BERT.

Em resumo, o código define um modelo de linguagem BERT e carrega um tokenizer associado a esse modelo, que será usado para processar o texto de entrada.

**Função `transform_labels`**

**Descrição**

A função `transform_labels` é responsável por transformar as etiquetas de saída do modelo em um formato adequado para o treinamento.

**Parâmetros**

* `label`: um dicionário que contém as etiquetas de saída do modelo.

**Retorno**

* Um dicionário com uma única chave `"label"` que contém uma lista de etiquetas transformadas.

**Funcionamento**

1. A função recebe um dicionário `label` como entrada.
2. Ela extrai o valor da chave `"output"` do dicionário `label` e o atribui à variável `label`.
3. Ela cria uma lista vazia `result` que será usada para armazenar as etiquetas transformadas.
4. Ela itera sobre cada etiqueta `l` na lista `label` e aplica a transformação usando o dicionário `mapDict`.
5. A transformação é feita substituindo cada etiqueta `l` pelo seu valor correspondente em `mapDict`.
6. A etiqueta transformada é adicionada à lista `result`.
7. A função retorna um dicionário com uma única chave `"label"` que contém a lista de etiquetas transformadas.

**Exemplo**

Suponha que o dicionário `label` seja:
```python
label = {"output": ["No Hate Speech", "Hate Speech", "No Hate Speech"]}
```
E que o dicionário `mapDict` seja:
```python
mapDict = {"No Hate Speech": 0, "Hate Speech": 1}
```
A função `transform_labels` retornará:
```python
{"label": [0, 1, 0]}
```
**Observações**

* A função assume que o dicionário `label` tenha uma chave `"output"` que contém uma lista de etiquetas.
* A função assume que o dicionário `mapDict` seja definido anteriormente e contenha as transformações necessárias.
* A função pode ser usada para transformar etiquetas de saída de modelos de linguagem em um formato adequado para o treinamento.



**Função `tokenize_function`**

**Descrição**

A função `tokenize_function` é responsável por tokenizar o texto de entrada usando um tokenizer pré-treinado.

**Parâmetros**

* `example`: um dicionário que contém o texto de entrada a ser tokenizado.

**Retorno**

* Um objeto de tokenização que contém os tokens gerados a partir do texto de entrada.

**Funcionamento**

1. A função recebe um dicionário `example` como entrada.
2. Ela extrai o valor da chave `"input"` do dicionário `example` e o passa como argumento para o tokenizer.
3. O tokenizer é chamado com os seguintes parâmetros:
	* `example["input"]`: o texto de entrada a ser tokenizado.
	* `padding=True`: indica que o tokenizer deve adicionar padding aos tokens para que todos tenham o mesmo comprimento.
	* `truncation=True`: indica que o tokenizer deve truncar o texto de entrada se ele for muito longo.
4. O tokenizer gera os tokens a partir do texto de entrada e os retorna como um objeto de tokenização.
5. A função retorna o objeto de tokenização gerado.

**Exemplo**

Suponha que o dicionário `example` seja:
```python
example = {"input": "Este é um exemplo de texto a ser tokenizado"}
```
E que o tokenizer seja um tokenizer pré-treinado como o BERT. A função `tokenize_function` retornará um objeto de tokenização que contém os tokens gerados a partir do texto de entrada, como por exemplo:
```python
{
  "input_ids": [101, 202, 303, 404, 505, 606, 707, 808, 909],
  "attention_mask": [1, 1, 1, 1, 1, 1, 1, 1, 1],
  "token_type_ids": [0, 0, 0, 0, 0, 0, 0, 0, 0]
}
```
**Observações**

* A função assume que o dicionário `example` tenha uma chave `"input"` que contém o texto de entrada a ser tokenizado.
* A função assume que o tokenizer seja pré-treinado e configurado para tokenizar o texto de entrada.
* A função pode ser usada para tokenizar texto de entrada em um formato adequado para o treinamento de modelos de linguagem.

In [6]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
tokenized_datasets

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

Map:   0%|          | 0/114 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'output', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 453
    })
    test: Dataset({
        features: ['input', 'output', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
        num_rows: 114
    })
})



**Código de pré-processamento de dados**

Este código é responsável por pré-processar os dados de treinamento e teste para serem utilizados em um modelo de linguagem.

**Linhas de código**

1. `tokenized_datasets = dataset.map(tokenize_function, batched=True)`

Esta linha de código aplica a função `tokenize_function` a cada exemplo no conjunto de dados `dataset`. A função `tokenize_function` é responsável por tokenizar o texto de entrada usando um tokenizer pré-treinado. O parâmetro `batched=True` indica que a função deve ser aplicada em lotes, o que pode melhorar a eficiência do processamento.

2. `tokenized_datasets = tokenized_datasets.map(transform_labels, batched=True)`

Esta linha de código aplica a função `transform_labels` a cada exemplo no conjunto de dados `tokenized_datasets`. A função `transform_labels` é responsável por transformar as etiquetas de saída do modelo em um formato adequado para o treinamento. O parâmetro `batched=True` indica que a função deve ser aplicada em lotes.

3. `data_collator = DataCollatorWithPadding(tokenizer=tokenizer)`

Esta linha de código cria um objeto `DataCollatorWithPadding` que é responsável por agregar os dados de entrada e saída em um formato adequado para o treinamento. O parâmetro `tokenizer=tokenizer` indica que o objeto deve usar o mesmo tokenizer pré-treinado utilizado anteriormente.

4. `tokenized_datasets`

Esta linha de código simplesmente retorna o conjunto de dados `tokenized_datasets` que foi pré-processado anteriormente.

**Objetivo**

O objetivo deste código é pré-processar os dados de treinamento e teste para serem utilizados em um modelo de linguagem. O código aplica as funções de tokenização e transformação de etiquetas aos dados de entrada e saída, e agrega os dados em um formato adequado para o treinamento.

**Observações**

* O código assume que o conjunto de dados `dataset` contenha os dados de treinamento e teste.
* O código assume que o tokenizer pré-treinado seja configurado corretamente para tokenizar o texto de entrada.
* O código pode ser utilizado como parte de um pipeline de pré-processamento de dados para modelos de linguagem.

### Treinando e Validando

Neste trecho, explicamos como utilizar o Transformers para treinar um modelo de classificação de sequências. Começamos definindo os argumentos de treinamento, como o diretório de saída e os parâmetros do treinamento. Importamos o Trainer e o modelo de classificação. Definimos uma função de métrica e criamos o objeto Trainer para treinar o modelo. Por fim, avaliamos o modelo treinado. O modelo obteve uma precisão de 68 e uma acurácia de 69.2.

In [7]:
output_dir = "./bert-hate-speech-test"

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    logging_dir="./logs.log",
    logging_steps=100,
    eval_strategy="steps",
    eval_steps=200,
    save_total_limit=2,
    save_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",
)



**Definição de argumentos de treinamento**

Este código define os argumentos de treinamento para um modelo de linguagem BERT que será treinado para detectar discurso de ódio.

**Argumentos de treinamento**

* `output_dir`: o diretório onde os resultados do treinamento serão salvos. Neste caso, o diretório é `./bert-hate-speech-test`.
* `num_train_epochs`: o número de épocas de treinamento. Neste caso, o modelo será treinado por 3 épocas.
* `per_device_train_batch_size`: o tamanho do lote de treinamento por dispositivo. Neste caso, o tamanho do lote é 8.
* `per_device_eval_batch_size`: o tamanho do lote de avaliação por dispositivo. Neste caso, o tamanho do lote é 8.
* `weight_decay`: a taxa de decadência de peso. Neste caso, a taxa é 0,01.
* `logging_dir`: o diretório onde os logs de treinamento serão salvos. Neste caso, o diretório é `./logs.log`.
* `logging_steps`: a frequência com que os logs de treinamento serão salvos. Neste caso, os logs serão salvos a cada 100 passos.
* `eval_strategy`: a estratégia de avaliação. Neste caso, a estratégia é `steps`, o que significa que o modelo será avaliado a cada 200 passos.
* `eval_steps`: a frequência com que o modelo será avaliado. Neste caso, o modelo será avaliado a cada 200 passos.
* `save_total_limit`: o número máximo de modelos que serão salvos. Neste caso, o número máximo é 2.
* `save_steps`: a frequência com que o modelo será salvo. Neste caso, o modelo será salvo a cada 200 passos.
* `load_best_model_at_end`: uma flag que indica se o melhor modelo deve ser carregado no final do treinamento. Neste caso, a flag é `True`.
* `metric_for_best_model`: a métrica que será usada para determinar o melhor modelo. Neste caso, a métrica é `accuracy`.
* `report_to`: o sistema de relatórios que será usado. Neste caso, o sistema é `none`, o que significa que não será usado nenhum sistema de relatórios.

**Objetivo**

O objetivo deste código é definir os argumentos de treinamento para um modelo de linguagem BERT que será treinado para detectar discurso de ódio. Os argumentos de treinamento são importantes para controlar o processo de treinamento e garantir que o modelo seja treinado de forma eficiente e eficaz.

In [8]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)

os.environ["WANDB_DISABLE"] = "true"
os.environ["WANDB_MODE"] = "offline"

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




**Carregamento do modelo e configuração do Weights & Biases**

Este código carrega um modelo de linguagem pré-treinado e configura o Weights & Biases para não enviar dados de treinamento para o servidor.

**Linhas de código**

1. `model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=3)`

Esta linha de código carrega um modelo de linguagem pré-treinado chamado `checkpoint` e o configura para realizar classificação de sequências com 3 rótulos. O modelo é carregado usando a classe `AutoModelForSequenceClassification` do Hugging Face Transformers.

2. `os.environ["WANDB_DISABLE"] = "true"`

Esta linha de código define uma variável de ambiente chamada `WANDB_DISABLE` e a configura como `true`. Isso desativa o Weights & Biases e impede que os dados de treinamento sejam enviados para o servidor.

3. `os.environ["WANDB_MODE"] = "offline"`

Esta linha de código define uma variável de ambiente chamada `WANDB_MODE` e a configura como `"offline"`. Isso configura o Weights & Biases para funcionar no modo offline, o que significa que os dados de treinamento serão armazenados localmente e não serão enviados para o servidor.

**Objetivo**

O objetivo deste código é carregar um modelo de linguagem pré-treinado e configurar o Weights & Biases para não enviar dados de treinamento para o servidor. Isso é útil quando você deseja treinar um modelo localmente e não deseja compartilhar os dados de treinamento com o servidor do Weights & Biases.

**Observações**

* O modelo carregado é um modelo de linguagem pré-treinado chamado `checkpoint`.
* O modelo é configurado para realizar classificação de sequências com 3 rótulos.
* O Weights & Biases é desativado e configurado para funcionar no modo offline.

In [9]:
metric = evaluate.load("accuracy")

def compute_metric(eval_perd):
  logits, labels = eval_perd
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

**1. `metric = evaluate.load("accuracy")`**

Aqui, você está carregando um objeto de métrica chamado `accuracy` usando a função `evaluate.load()`. A função `evaluate` é provavelmente parte de uma biblioteca de avaliação de modelos, como o TensorFlow ou o Hugging Face Transformers. O objeto `metric` será usado para calcular a precisão do modelo.

**2. `def compute_metric(eval_perd):`**

Aqui, você define uma função chamada `compute_metric` que recebe um argumento `eval_perd`. Essa função será responsável por calcular a precisão do modelo.

**3. `logits, labels = eval_perd`**

Dentro da função `compute_metric`, você desempacota o argumento `eval_perd` em duas variáveis: `logits` e `labels`. `logits` são as saídas brutas do modelo (antes da aplicação da função de ativação), enquanto `labels` são as labels verdadeiras dos dados.

**4. `predictions = np.argmax(logits, axis=-1)`**

Aqui, você usa a função `np.argmax` para obter as previsões do modelo. `np.argmax` retorna o índice do maior valor em cada linha da matriz `logits`. O argumento `axis=-1` especifica que você deseja considerar a última dimensão da matriz (ou seja, a dimensão das classes). As previsões são armazenadas na variável `predictions`.

**5. `return metric.compute(predictions=predictions, references=labels)`**

Finalmente, você usa o objeto `metric` para calcular a precisão do modelo. A função `compute` recebe duas argumentos: `predictions` (as previsões do modelo) e `references` (as labels verdadeiras). O resultado é retornado pela função `compute_metric`.

Em resumo, essa função calcula a precisão do modelo comparando as previsões com as labels verdadeiras.

In [10]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metric
)

C:\Users\Usuário\AppData\Local\Temp\ipykernel_5204\202270960.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


**1. `Trainer`**

`Trainer` é uma classe que provavelmente faz parte da biblioteca Hugging Face Transformers. Ela é responsável por treinar um modelo de linguagem.

**2. `model`**

`model` é o modelo de linguagem que será treinado. Ele provavelmente é uma instância de uma classe que herda de `PreTrainedModel` ou `TFPreTrainedModel`.

**3. `training_args`**

`training_args` é um objeto que contém os argumentos de treinamento para o modelo. Ele provavelmente inclui parâmetros como o número de épocas, a taxa de aprendizado, o tamanho do lote, etc.

**4. `train_dataset` e `eval_dataset`**

`train_dataset` e `eval_dataset` são os conjuntos de dados de treinamento e avaliação, respectivamente. Eles são instâncias de `Dataset` e contêm os dados que serão usados para treinar e avaliar o modelo. Nesse caso, os conjuntos de dados são `tokenized_datasets["train"]` e `tokenized_datasets["test"]`, o que sugere que os dados foram pré-processados e tokenizados.

**5. `data_collator`**

`data_collator` é um objeto que é responsável por agrupar os dados em lotes para o treinamento. Ele provavelmente é uma instância de `DataCollator`.

**6. `tokenizer`**

`tokenizer` é o tokenizador que foi usado para pré-processar os dados. Ele provavelmente é uma instância de `Tokenizer`.

**7. `compute_metrics`**

`compute_metrics` é a função que será usada para calcular as métricas de avaliação do modelo. Nesse caso, é a função `compute_metric` que foi definida anteriormente.

Ao criar uma instância de `Trainer`, você está configurando o treinamento do modelo com os parâmetros e conjuntos de dados especificados. O `Trainer` será responsável por executar o treinamento e avaliação do modelo.

In [11]:
trainer.train()

  0%|          | 0/171 [00:00<?, ?it/s]

{'loss': 0.6485, 'grad_norm': 8.408008575439453, 'learning_rate': 2.0760233918128656e-05, 'epoch': 1.75}
{'train_runtime': 881.9495, 'train_samples_per_second': 1.541, 'train_steps_per_second': 0.194, 'train_loss': 0.5785766289248104, 'epoch': 3.0}


TrainOutput(global_step=171, training_loss=0.5785766289248104, metrics={'train_runtime': 881.9495, 'train_samples_per_second': 1.541, 'train_steps_per_second': 0.194, 'total_flos': 47489916326328.0, 'train_loss': 0.5785766289248104, 'epoch': 3.0})

In [12]:
trainer.evaluate()

  0%|          | 0/15 [00:00<?, ?it/s]

{'eval_loss': 0.6757541298866272,
 'eval_accuracy': 0.6929824561403509,
 'eval_runtime': 12.2417,
 'eval_samples_per_second': 9.312,
 'eval_steps_per_second': 1.225,
 'epoch': 3.0}

### Salvando e utilizando o modelo

Neste resumo, explicamos como salvar um modelo no Hugging Face, fazer o push para o repositório e importá-lo usando a pipeline do Transformers. O instrutor demonstrou como testar o modelo com um texto de exemplo e interpretar os resultados. Ele enfatizou a importância do fine tuning de modelos de linguagem para diferentes casos de uso, como identificar discursos de ódio. Essa abordagem é preferível a treinar um novo modelo do zero.

In [13]:
trainer.save_model()

In [14]:
trainer.push_to_hub("devmgck/modelhate")

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.24k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/devmgck/bert-hate-speech-test/commit/737be13fe0178bd87434b2804a92f1f61ec8915a', commit_message='devmgck/modelhate', commit_description='', oid='737be13fe0178bd87434b2804a92f1f61ec8915a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/devmgck/bert-hate-speech-test', endpoint='https://huggingface.co', repo_type='model', repo_id='devmgck/bert-hate-speech-test'), pr_revision=None, pr_num=None)

In [15]:
# Use a pipeline as a high-level helper
pipe = pipeline("text-classification", model="devmgck/bert-hate-speech-test")

config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

c:\Users\Usuário\.virtualenvs\ia_para_devs-NJoqudvH\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Usuário\.cache\huggingface\hub\models--devmgck--bert-hate-speech-test. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]



Aqui está uma explicação detalhada do código:

**1. `pipeline`**

`pipeline` é uma função responsável por criar uma pipeline de processamento de texto que pode ser usada para realizar tarefas específicas, como classificação de texto.

**2. `"text-classification"`**

O primeiro argumento da função `pipeline` é o tipo de pipeline que será criada. Nesse caso, é `"text-classification"`, o que significa que a pipeline será usada para classificar texto em diferentes categorias.

**3. `model="devmgck/bert-hate-speech-test"`**

O segundo argumento da função `pipeline` é o modelo que será usado para realizar a classificação de texto. Nesse caso, é o modelo `devmgck/bert-hate-speech-test`, que é um modelo pré-treinado que foi treinado para detectar discurso de ódio em texto.

O modelo `devmgck/bert-hate-speech-test` é um modelo BERT (Bidirectional Encoder Representations from Transformers) que foi treinado em um conjunto de dados de texto que contém exemplos de discurso de ódio. O modelo é capaz de classificar texto em diferentes categorias, como "ódio" ou "não ódio".

Ao criar uma pipeline com o modelo `devmgck/bert-hate-speech-test`, você está configurando uma ferramenta que pode ser usada para detectar discurso de ódio em texto. A pipeline pode ser usada para analisar texto e retornar uma classificação que indica se o texto contém discurso de ódio ou não.

In [ ]:
pipe(inputs="Que legal! Eu sou uma cotista da flip de um delta 36 em Paraty. Recomendo!", truncation=True)

[{'label': 'LABEL_0', 'score': 0.7875879406929016}]